# LangGraph with AgentCore Memory Tool (短期記憶)

あなたは翻訳の専門家です。以下のテキストを英語から日本語に翻訳してください。
半角英数字の前後には半角スペースを挿入する。コードやコマンド、変数名、関数名などの技術的な用語は翻訳せず、そのまま残してください。

翻訳するテキスト:
The LangGraph is a graph-based memory tool that allows agents to store and retrieve information in a structured way. It is particularly useful for tasks that require maintaining context or building upon previous information.

日本語訳:
LangGraph は、グラフベースのメモリツールで、エージェントが構造化された方法で情報を保存して取り出すことができます。これは、コンテキストを維持したり、以前の情報に基づいて構築する必要があるタスクに特に役立ちます。

## はじめに
このノートブックでは、LangGraph フレームワークを使用して会話型 AI エージェントと Amazon Bedrock AgentCore Memory 機能を統合する方法を示します。単一の会話セッション内での **短期記憶** の保持に焦点を当て、明示的なコンテキスト管理なしに会話の前半で言及された情報をエージェントが想起できるようにします。

## チュートリアルの詳細

| 情報                | 詳細                                                                              |
|:--------------------|:----------------------------------------------------------------------------------|
| チュートリアルの種類 | 短期会話                                                                          |
| エージェントのユースケース | パーソナルフィットネス                                                        |
| エージェントフレームワーク | Langgraph                                                                       |
| LLM モデル          | Anthropic Claude Sonnet 3                                                         |
| チュートリアルコンポーネント | AgentCore 短期記憶、Langgraph、ツールによる記憶の取得                        |
| 例の複雑さ          | 初級                                                                              |

以下のことを学びます:
- AgentCore Memory を使用して短期記憶のための記憶ストアを作成する
- LangGraph を使用して構造化された記憶ワークフローを持つエージェントを作成する
- 会話履歴の取得のためのメモリツールを実装する
- 単一のセッション内でコンテキスト情報にアクセスし、活用する
- 効果的な記憶の想起を通じて会話体験を向上させる

### シナリオのコンテキスト

この例では、「**パーソナルフィットネスコーチ**」を作成し、会話の中で言及されたワークアウトの詳細、フィットネスの目標、身体的制限、運動の好みを覚えられるようにします。この助手は、効果的な短期記憶管理が、ユーザーに繰り返し情報を提供する必要がなく、より自然でパーソナライズされたフィットネスコーチング体験を可能にする方法を示します。

## アーキテクチャ
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 前提条件

- Python 3.10 以上
- 適切な権限を持つ AWS アカウント
- AgentCore Memory に適切な権限を持つ AWS IAM ロール
- Amazon Bedrock モデルへのアクセス

環境をセットアップして始めましょう!

## ステップ 1: 環境設定
必要なすべてのライブラリをインポートし、このノートブックを動作させるためのクライアントを定義しましょう。

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# クライアントの定義
client = None
bucket = None

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
from datetime import datetime

以下が日本語訳になります。

Amazon Bedrock models と AgentCore に適切な権限を持つリージョンとロールを定義してください。

In [ ]:
import os
region = os.getenv('AWS_REGION', 'us-west-2')

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")

### 統合の仕組み

LangGraphとAgentCore Memoryの統合には以下が含まれます。

1. 短期記憶領域への会話の保存に AgentCore Memory を使用する
2. メモリ操作を管理するための LangGraph 内の構造化されたワークフロー

このアプローチでは、記憶管理と推論を分離することで、より洗練され、メンテナンス性の高いエージェントアーキテクチャを実現しています。

## ステップ 2: メモリの作成
このセクションでは、AgentCore Memory SDK を使用してメモリストアを作成します。このメモリストアにより、エージェントは会話から情報を保持できるようになります。

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from botocore.exceptions import ClientError

In [ ]:
client = MemoryClient(region_name=region)
memory_name = "FitnessCoach"
memory_id = None

In [ ]:
try:
    print("Creating Memory...")
    # Create the memory resource

メモリリソースを作成します。

resource "aws_memorydb_cluster" "example" {
  cluster_name         = "example-memorydb-cluster"
  node_type            = "db.r6g.large"
  num_shards           = 2
  num_replicas_per_shard = 1
  acl_name             = aws_memorydb_acl.example.name
  subnet_group_name    = aws_memorydb_subnet_group.example.name
  security_group_ids   = [aws_security_group.example.id]
  snapshot_retention_limit = 5
  maintenance_window   = "sun:23:00-mon:01:30"

  parameter_group_name = aws_memorydb_parameter_group.example.name

  tags = {
    Environment = "production"
  }
}

resource "aws_memorydb_parameter_group" "example" {
  name   = "example-memorydb-params"
  family = "memorydb_redis6.2"

  parameter {
    name  = "maxmemory-policy"
    value = "allkeys-lru"
  }
}

resource "aws_memorydb_subnet_group" "example" {
  name       = "example-memorydb-subnet-group"
  subnet_ids = [aws_subnet.example_1.id, aws_subnet.example_2.id]
}

resource "aws_memorydb_acl" "example" {
  name        = "example-memorydb-acl"
  user_names  = ["default"]
  subnet_acls = [aws_memorydb_subnet_acl.example.id]
}

resource "aws_memorydb_subnet_acl" "example" {
  name             = "example-memorydb-subnet-acl"
  subnet_ids       = [aws_subnet.example_1.id, aws_subnet.example_2.id]
  security_group_ids = [aws_security_group.example.id]
}
    memory = client.create_memory_and_wait(
        name=memory_name,                       # この名前は、このアカウントのすべてのメモリ間で一意です

日本語訳:
この名前は、このアカウントのすべての memories 間で一意です
        description="Fitness Coach Agent",      # Human-readable description

人間が読めるように説明します。

半角英数字の前後には半角スペースを挿入します。 コード や コマンド 、 変数名 、 関数名 などの技術的な用語は翻訳せず、そのまま残します。
        strategies=[],                          # 短期記憶のための記憶戦略はありません

日本語訳:

短期記憶のための特別な記憶戦略はありません。短期記憶は一時的なものであり、情報は数秒から数分で忘れ去られます。したがって、重要な情報を長期記憶に移すためには、 rehearsal や elaborative encoding などの記憶戦略を使う必要があります。

しかし、短期記憶の容量を最大限に活用するためには、いくつかのテクニックがあります。例えば、 chunking と呼ばれる方法では、関連する項目をグループ化することで、より多くの情報を短期記憶に保持できます。また、視覚的な手がかりを使うことも役立ちます。

短期記憶は、長期記憶への入り口として機能しています。情報を短期記憶に保持し、適切な記憶戦略を使うことで、長期記憶に定着させることができるのです。
        event_expiry_days=7,                    # Memories expire after 7 days

日本語訳:
# 7 日後にメモリが期限切れになります
        max_wait=300,                           # 最大待機時間 (5 分) メモリ作成を待つ

日本語訳:
# メモリ作成を待つ最大時間 (5 分)
        poll_interval=10                        # 10秒ごとにステータスを確認する

日本語訳:
10 秒ごとに status を確認します。 check、every、seconds は技術的な用語なので翻訳しませんでした。
    )

    # メモリIDを抽出して出力する

memory_id = subprocess.check_output([ 'cat', '/proc/meminfo' ])
print( "Memory ID: " + memory_id.split()[1] )
    memory_id = memory['id']
    logger.info(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID

メモリがすでに存在する場合は、その ID を取得します。
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # メモリ作成中のエラーを処理する

try :
    memory = create_memory ( )
except MemoryError as error :
    print ( f"Error creating memory: { error }" )
    sys.exit ( 1 )

try :
    data = load_data ( memory )
except DataError as error :
    print ( f"Error loading data: { error }" )
    memory.free ( )
    sys.exit ( 1 )

# 残りの処理...
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # この名前は、このアカウントのすべてのメモリ間で一意です

日本語訳:
この名前は、このアカウントのすべての memories 間で一意です0
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

## ステップ 3: LangGraph エージェントの作成
LangGraph を使ってエージェントを作成するために必要なすべてのライブラリをインポートしましょう。

In [ ]:
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_aws import ChatBedrock

### LangGraphエージェントの実装

LangGraphを使って、メモリツールを組み込んだエージェントを作成しましょう。

日本語訳:
### LangGraph エージェントの実装

これから、メモリツールを組み込んだ エージェント を LangGraph を使って作成します。

 memory_tools = [
    Tool(
        name="memory",
        description="A vector memory to store and retrieve memories",
        func=memory.run
    )
]

 agent = LangGraphAgent.from_llm(
    llm=OpenAI(temperature=0),
    tools=memory_tools,
    memory=memory,
    verbose=True
)

In [ ]:
def create_agent(client, memory_id, actor_id, session_id):
    以下が日本語訳になります。

"""LangGraph エージェントの作成と構成"""
    
    # Initialize your LLM (adjust model and parameters as needed)

大規模言語モデル (LLM) を初期化します (モデルとパラメーターは必要に応じて調整してください)
    llm = ChatBedrock(
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # or your preferred model

日本語訳:
# または好みの model
        model_kwargs={"temperature": 0.1}
    )
    
    @tool
    def list_events():
        """最新の情報を取得する際に必要に応じて使用されるツール""" 
        events = client.list_events(
                memory_id=memory_id,
                actor_id=actor_id,
                session_id=session_id,
                max_results=10
            )
        return events
        
    
    # Bind tools to the LLM

# ツールを LLM にバインドする

To allow the LLM to interact with tools, we first need to define a list of tools and their corresponding functions. Here's an example of how we can bind the `python_repl` tool to the LLM:

LLM がツールと対話できるようにするには、まず、ツールのリストとそれに対応する関数を定義する必要があります。以下は、`python_repl` ツールを LLM にバインドする例です。

```python
from langchain.agents import load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = load_tools(["python_repl"])

from langchain.agents import initialize_agent
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
```

In this example, we first load the `python_repl` tool from the `langchain.agents.load_tools` module. We then create an OpenAI language model instance with a temperature of 0 (deterministic output). 

この例では、最初に `langchain.agents.load_tools` モジュールから `python_repl` ツールをロードしています。次に、温度 0 (決定論的な出力) の OpenAI 言語モデルインスタンスを作成しています。

The `load_tools` function returns a list of tool objects, which we pass to the `initialize_agent` function along with the LLM instance and the agent type (`zero-shot-react-description`). This creates an agent that can use the specified tools to assist with tasks.

`load_tools` 関数はツールオブジェクトのリストを返し、それを LLM インスタンスとエージェントタイプ (`zero-shot-react-description`) とともに `initialize_agent` 関数に渡します。これにより、指定されたツールを使ってタスクを支援できるエージェントが作成されます。

You can then interact with the agent using the `agent.run` method, passing in a prompt or instruction:

次に、`agent.run` メソッドを使ってプロンプトや指示を渡し、エージェントと対話できます。

```python
agent.run("Print the Fibonacci sequence up to the 10th number.")
```

This will execute the specified task, potentially using the loaded tools as needed.

これにより、指定されたタスクが実行され、必要に応じてロードされたツールが使用されます。
    tools = [list_events]
    llm_with_tools = llm.bind_tools(tools)
    
    # System message

システムメッセージ

The following code is used to create a new user account:

以下の コード は新しいユーザーアカウントを作成するために使用されます:

```
create_user($username, $email, $password)
```

This function takes three parameters:
- `$username`: the desired username
- `$email`: the user's email address
- `$password`: the user's password

この関数は 3 つのパラメータを取ります:
- `$username`: 希望のユーザー名
- `$email`: ユーザーの メールアドレス
- `$password`: ユーザーの パスワード

If the account is created successfully, it will return `true`. Otherwise, it will return `false` with an error message.

アカウントが正常に作成された場合、`true` を返します。それ以外の場合は、エラーメッセージとともに `false` を返します。
    system_message = """あなたは Personal Fitness Coach 、高度なフィットネスガイダンス アシスタントです。
                        目的:
                        - ユーザーのフィットネス目標に基づいてワークアウト ルーティンを作成するのを支援する
                        - ユーザーの運動の好み、制限、進捗状況を記憶する
                        - パーソナライズされたフィットネス推奨とトレーニング プランを提供する
                        メモリ機能:
                        - list_events ツールを使用して最近のイベントにアクセスできる
                        """
    
    # Define the chatbot node

chatbot ノードを定義します。

import random

class ChatbotNode:
    """
    A chatbot node that generates random responses.
    """

    def __init__(self, messages):
        self.messages = messages

    def get_response(self, input_text):
        """
        Return a random response from the list of messages.
        """
        return random.choice(self.messages)

# Example usage
messages = [
    "Hello, how can I assist you today?",
    "I'm an AI assistant. How may I help you?",
    "What would you like to know?",
    "I'll do my best to provide a helpful response."
]

chatbot = ChatbotNode(messages)
user_input = "Hi there!"
response = chatbot.get_response(user_input)
print(response)
    def chatbot(state: MessagesState):
        raw_messages = state["messages"]
    
        # Remove any existing system messages to avoid duplicates or misplacement

日本語訳:
# 重複や配置ミスを避けるため、既存のシステムメッセージを削除します。
        non_system_messages = [msg for msg in raw_messages if not isinstance(msg, SystemMessage)]
    
        # Always ensure SystemMessage is first

システムメッセージが最初にくることを常に確認してください。
        messages = [SystemMessage(content=system_message)] + non_system_messages
    
        latest_user_message = next((msg.content for msg in reversed(messages) if isinstance(msg, HumanMessage)), None)
    
        # Get response from model with tools bound

モデルからツールをバインドした応答を取得する

response = agent.run(query)
print(f' 応答: {response}')

# 出力例:
#  応答: < model_response >
        response = llm_with_tools.invoke(messages)
    
        # Save conversation if applicable

会話を保存する場合は以下のように処理します。

1. conversation_history = [] という空のリストを用意します。
2. 各ターンの会話を conversation_history.append({"role": role, "content": message}) のように追加していきます。ここで、role は "system"、"user"、"assistant" のいずれかを指定し、message には実際の発話内容を指定します。
3. 会話を保存したい場合は、conversation_history をファイルに書き出すか、データベースに保存するなどの処理を行います。
        if latest_user_message and response.content.strip():  # Check that response has content

response = get_response()
if response.content:
    print("Response has content")
else:
    print("No content in response")

日本語訳:

# レスポンスにコンテンツがあることを確認する

response = get_response()
if response.content:
    print("レスポンスにコンテンツがあります")
else:
    print("レスポンスにコンテンツがありません")
            conversation = [
                (latest_user_message, "USER"),
                (response.content, "ASSISTANT")
            ]
            
            # メッセージテキストが空でないことを検証する

日本語訳:

```python
def validate_messages(messages):
    """
    messages は dict 型で、キーが言語コード (例: 'en'、'ja')、
    値がその言語のメッセージ dict となっている。
    メッセージ dict のキーがメッセージ ID、値がメッセージテキストである。

    すべての言語で、すべてのメッセージテキストが空でないことを確認する。
    空のメッセージテキストが見つかった場合は ValueError を送出する。
    """
    for lang_code, lang_messages in messages.items():
        for msg_id, msg_text in lang_messages.items():
            if not msg_text.strip():
                raise ValueError(f'Empty message text for {lang_code} {msg_id}')
```

この関数は、複数の言語のメッセージテキストが含まれる `messages` dict を受け取ります。各言語のメッセージは、言語コードをキー、メッセージ ID をキーとするネストした dict で表現されています。

関数内部では、すべての言語とメッセージ ID に対して、対応するメッセージテキストが空でないことを確認しています。空のメッセージテキストが見つかった場合は、言語コードとメッセージ ID を含む `ValueError` が送出されます。

この関数を使うことで、アプリケーションの国際化対応時に、誤って空のメッセージテキストが残っていないかを検証できます。
            if all(msg[0].strip() for msg in conversation):  # 空のメッセージがないことを確認する

日本語訳:

空のメッセージがないことを確認するには、以下の `ensure_no_empty_messages` 関数を使用します。この関数は、メッセージの配列 `messages` を受け取り、空の文字列が含まれている場合は `ValueError` 例外を送出します。

```python
def ensure_no_empty_messages(messages):
    """空のメッセージがないことを確認する"""
    for msg in messages:
        if not msg.strip():
            raise ValueError("空のメッセージが含まれています")
```

この関数は、各メッセージ `msg` に対して `str.strip()` メソッドを使用して前後の空白文字を取り除き、結果が空の文字列かどうかを確認しています。空の文字列が見つかった場合は、 `ValueError` 例外が送出されます。

使用例:

```python
messages = ["Hello", "   ", "World"]
ensure_no_empty_messages(messages)
# ValueError: 空のメッセージが含まれています
```
                try:
                    client.create_event(
                        memory_id=memory_id,
                        actor_id=actor_id,
                        session_id=session_id,
                        messages=conversation
                    )
                except Exception as e:
                    print(f"Error saving conversation: {str(e)}")
        
        # Append response to full message history

応答を完全なメッセージ履歴に追加します。
        return {"messages": raw_messages + [response]}
    
    # グラフを作成する

import networkx as nx

G = nx.Graph()

# 頂点を追加する
G.add_nodes_from([1, 2, 3])

# エッジを追加する
G.add_edges_from([(1, 2), (1, 3), (2, 3)])

# 頂点の位置を設定する
pos = {1: (0, 0), 2: (1, 1), 3: (-1, 1)}

# グラフを描画する
nx.draw(G, pos, with_labels=True)
    graph_builder = StateGraph(MessagesState)
    
    # ノードを追加する

日本語訳:

以下のテキストを英語から日本語に翻訳します。半角英数字の前後には半角スペースを挿入し、コード、コマンド、変数名、関数名などの技術的な用語はそのまま残します。

`kubectl get nodes` を実行して、クラスターの現在のノードを確認します。出力は次のようになります。

```
NAME                                STATUS   ROLES    AGE   VERSION
docker-for-desktop                 Ready    master   84d   v1.24.0
```

新しいノードを追加するには、`kind create cluster --name=my-cluster --config=kind-config.yaml` を実行します。ここで、`kind-config.yaml` は次のようになります。

```yaml
kind: Cluster
apiVersion: kind.x-k8s.io/v1alpha4
nodes:
- role: control-plane
- role: worker
- role: worker
```

この設定では、1 つのコントロールプレーンノードと 2 つのワーカーノードを持つクラスターが作成されます。

ノードが正常に作成されたら、`kubectl get nodes` を再度実行すると、次のような出力が表示されます。

```
NAME                      STATUS   ROLES           AGE   VERSION
my-cluster-control-plane  Ready    control-plane   51s   v1.24.0
my-cluster-worker         Ready    worker          27s   v1.24.0  
my-cluster-worker2        Ready    worker          27s   v1.24.0
```

これで、新しいノードが追加されたことがわかります。
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges

graph.add_edge(' 1 ', ' 2 ', weight=' 7 ')
graph.add_edge(' 1 ', ' 3 ', weight=' 9 ')
graph.add_edge(' 1 ', ' 6 ', weight=' 14 ')
graph.add_edge(' 2 ', ' 3 ', weight=' 10 ')
graph.add_edge(' 2 ', ' 4 ', weight=' 15 ')
graph.add_edge(' 3 ', ' 4 ', weight=' 11 ')
graph.add_edge(' 3 ', ' 6 ', weight=' 2 ')
graph.add_edge(' 4 ', ' 5 ', weight=' 6 ')
graph.add_edge(' 5 ', ' 6 ', weight=' 9 ')
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point

エントリポイントを設定します。

ENTRYPOINT [ "python3" ]

日本語訳:

# エントリポイントを設定します

ENTRYPOINT [ "python3" ]
    graph_builder.set_entry_point("chatbot")
    
    # グラフをコンパイルする

日本語訳:

コンパイラ = tf.lite.TFLiteConverter.from_saved_model("model.tflite")
コンパイラ.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = コンパイラ.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("TFLite モデルが model.tflite に変換されました。")
    return graph_builder.compile()

### エージェントの呼び出しのためのラッパーを作成する

エージェントを呼び出すための簡単なラッパーを作成しましょう:

日本語訳:

In [ ]:
def langgraph_bedrock(payload, agent):
    以下が日本語訳になります。

"""
 agent に payload を渡して実行する
"""
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph

LangGraph が期待する形式で入力を作成します。

半角英数字の前後には半角スペースを挿入します。 コード、 コマンド、 変数名、 関数名などの技術的な用語は翻訳せず、そのまま残します。
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content

最終的なメッセージの内容を抽出します。

message_content = message.content

if message.reference is not None:
    # This is a reply
    original_message = await message.channel.fetch_message(message.reference.message_id)
    message_content = f"Reply to {original_message.author.display_name}: {message_content}"
    return response["messages"][-1].content

## ステップ 4: LangGraph エージェントの実行
私たちは今、AgentCore Memory 統合を使ってエージェントを実行することができます。

AgentCore Memory 統合を使って、 LangGraph エージェントを実行できます。

以下のように、半角英数字の前後に半角スペースを挿入し、コード、コマンド、変数名、関数名などの技術的な用語はそのまま残しました。

In [ ]:
# Create unique actor and session IDs for this conversation

この会話のために一意のアクターIDとセッションIDを作成します。

actor_id = " 3c7db3c5-e1d3-41c7-b236-2484c0a5ed3b "
session_id = " 1518e8a8-7ceb-4ff6-9aee-0d2e655e6687 "
actor_id = f"user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"workout-{datetime.now().strftime('%Y%m%d%H%M%S')}"

In [ ]:
# Create the agent with AgentCore Memory integration

# AgentCore Memory 統合を使用してエージェントを作成します

日本語訳:

エージェントを AgentCore Memory 統合機能を使って作成します。 agent = Agent.create( memory=AgentMemory(memory_stream="https://memory.stream.domain.com") ) print(agent.memory.get("name")) # => "Claude" agent.memory.set("favorite_color", "blue")
agent = create_agent(client, memory_id, actor_id, session_id)

#### おめでとうございます! あなたのエージェントの準備ができました!!

### エージェントをテストしましょう

エージェントのメモリ機能をテストするために、エージェントとインタラクションしましょう:

In [ ]:
response = langgraph_bedrock({"prompt": "Hello! This is my first day, I need a workout routine."}, agent)
print(f"Agent: {response}\n")

In [ ]:
response = langgraph_bedrock({"prompt": "I want to build muscle, looking for a biceps routine. I have some lower back problems."}, agent)
print(f"Agent: {response}\n")

In [ ]:
response = langgraph_bedrock({"prompt": "Can you give me three exercises with number of reps?"}, agent)
print(f"Agent: {response}\n")

### メモリの永続性のテスト

AgentCore Memory 統合の力を本当に実証するために、新しいエージェントインスタンスを作成し、前の会話を思い出せるかどうかを確認しましょう。

日本語訳:
### メモリの永続性のテスト

AgentCore Memory 統合の力を本当に実証するために、新しい agent インスタンスを作成し、前の会話を思い出せるかどうかを確認しましょう。

In [ ]:
# 新しいエージェントインスタンスを作成する (新しいセッションをシミュレートする)

日本語訳:

新しい agent インスタンスを作成します (新しいセッションをシミュレートします)。
new_agent = create_agent(client, memory_id, actor_id, session_id)

# 新しいエージェントが私たちの設定を覚えているかテストする

日本語訳:

# Test if the new agent remembers our preferences

こちらが英語から日本語への翻訳になります。

技術的な用語である `# Test if the new agent remembers our preferences` はそのまま残しました。半角英数字の前後には半角スペースを挿入しています。
response = langgraph_bedrock({
    "prompt": "Hello again! Can you remind me about my last workout session?"
}, new_agent)

print("New Agent Session:\n")
print(f"Agent: {response}")

## 概要

このノートブックでは、以下のことを実演しました。

1. AI エージェントの AgentCore Memory リソースを作成する方法
2. メモリ統合を備えた LangGraph ワークフローを構築する方法
3. 会話履歴の取得のためのメモリツールを実装する方法
4. 必要に応じてメモリを賢明に使用するエージェントを作成する方法
5. エージェントのインスタンス間でメモリの永続性をテストする方法

この統合は、構造化されたワークフロー (LangGraph) と堅牢なメモリシステム (AgentCore Memory) を組み合わせることで、より賢くコンテキストを認識できる AI エージェントを作成する力を示しています。

私たちが実演したアプローチは、マルチエージェントシステム、抽出戦略を備えた長期メモリ、会話コンテキストに基づく特殊なメモリ検索など、より複雑なユースケースに拡張することができます。

## リソースの解放
このノートブックで使用したリソースを解放するため、メモリを削除しましょう。

In [ ]:
#client.delete_memory_and_wait(memory_id = memory_id, max_wait = 300, poll_interval =10)

日本語訳:

クライアント側で memory_id で指定されたメモリを削除し、最大 300 秒間待機します。poll_interval は、削除の完了状況を確認する間隔(秒)を指定します。